In [28]:
import tensorflow as tf
import numpy as np
import os
from distutils.version import LooseVersion
import warnings
from collections import Counter

In [29]:
import numpy as np
np.set_printoptions(threshold = 1e6)

In [30]:
dataset = 'sms'
trainingSection = 1

trainDataDir = '/Users/SamZhang/Documents/Capstone/dataset/' + dataset + '/train'
posTrain = trainDataDir + '/spam/' + str(trainingSection) + '/' + dataset + '_train.spam'
negTrain = trainDataDir + '/ham/' + str(trainingSection) + '/' + dataset + '_train.ham'

testDataDir = '/Users/SamZhang/Documents/Capstone/dataset/' + dataset + '/test'
posTest = testDataDir + '/spam/' + str(trainingSection) + '/' + dataset + '_test.spam'
negTest = testDataDir + '/ham/' + str(trainingSection) + '/' + dataset + '_test.ham'

saveDataPath = '/Users/SamZhang/Documents/Capstone/Models/runs/lstmmodel/' + str(trainingSection) + '/'

In [31]:
lstm_size = 256
lstm_layers = 2
batch_size = 128
learning_rate = 0.001
drop_out = 0.5
epochs = 30
embed_size = 128 
evaluate_every = 100
sequence_len = 200
split_frac = 0.8

# Processing data

In [32]:
def getTextLabel(posFilePath, negFilePath):
    import fileinput
    from string import punctuation

    labels = []
    texts = []
    for line in fileinput.input(posFilePath):
        line = line.lower()
        line = ''.join([c for c in line if c not in punctuation])
        if len(line) > 0:
            texts.append(line)
            labels.append(1)

    for line in fileinput.input(negFilePath):
        line = line.lower()
        line = ''.join([c for c in line if c not in punctuation])
        if len(line) > 0:
            texts.append(line)
            labels.append(0)
    labels = np.array(labels)
    
    return texts, labels

In [33]:
def getVocabToInt(texts):
    wordSet = ' '.join(texts).split()
    counts = Counter(wordSet)
    vocab = sorted(counts, key=counts.get, reverse = True)

    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    vocab_size = len(vocab_to_int)
    return vocab_to_int

In [34]:
def getFeatures(texts, labels, vocab_to_int):
    texts_in_int = []
    for line in texts:
        texts_in_int.append([vocab_to_int[word] for word in line.split()])

    text_lens = Counter([len(x) for x in texts_in_int])
    non_zero_idx = [ii for ii, texts in enumerate(texts_in_int) if len(texts) != 0] # all data that len > 0

    texts_in_int = [texts_in_int[ii] for  ii in non_zero_idx] #all sentences
    labels = np.array([labels[ii] for ii in non_zero_idx]) #0 for ham, 1 for spam
    features = np.zeros((len(texts_in_int), sequence_len), dtype=int)
    for i, row in enumerate(texts_in_int):
         features[i, -len(row):] = np.array(row)[:sequence_len]
    
    return features

In [35]:
import random
def shuffle(texts, labels):
    curSize = len(texts)
    for i in range(curSize):
        randIndex = random.randint(0, curSize - 1)
        tempText = texts[randIndex]
        texts[randIndex] = texts[i]
        texts[i] = tempText
        
        tempLabel = labels[randIndex]
        labels[randIndex] = labels[i]
        labels[i] = tempLabel

In [36]:
texts_train, labels_train = getTextLabel(posTrain, negTrain)
shuffle(texts_train, labels_train)
texts_test, labels_test = getTextLabel(posTest, negTest)

texts = texts_train + texts_test
vocab_to_int = getVocabToInt(texts)
vocab_size = len(vocab_to_int)

features_train = getFeatures(texts_train, labels_train, vocab_to_int)

# Seperate Training, Validation, Test set

In [37]:
split_idx = int(len(features_train)*split_frac)
train_x, val_x = features_train[:split_idx], features_train[split_idx:]
train_y, val_y = labels_train[:split_idx], labels_train[split_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape))

			Feature Shapes:
Train set: 		(446, 200) 
Validation set: 	(112, 200)


# Building LSTM 

In [38]:
# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [39]:
# Size of the embedding vectors (number of units in the embedding layer)
with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_size), -1, 1)) #generate random number from [-1, 1]
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [40]:
#building cell
with graph.as_default():
    #define lstm cell
    def lstm_cell():
        cell = tf.contrib.rnn.LSTMCell(lstm_size, 
                                       initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=2),
                                       state_is_tuple = True,
                                      reuse=tf.get_variable_scope().reuse)
        drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = keep_prob)
        return drop
    
    stack_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    initial_state = state = stack_cells.zero_state(batch_size, tf.float32)

In [41]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(stack_cells, embed, initial_state=initial_state)
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    
    cost = tf.losses.mean_squared_error(labels_, predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [42]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [43]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [44]:
def get_batches_test(text, x, y, batch_size=100):
    n_batches = len(x)//batch_size
    text, x, y = text[:n_batches*batch_size], x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield text[ii:ii+batch_size], x[ii:ii+batch_size], y[ii:ii+batch_size]

In [45]:
with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer()) 
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: drop_out,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%evaluate_every ==0:
                val_acc = []
                val_state = sess.run(stack_cells.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, saveDataPath + "sentiment.ckpt")

Epoch: 1/30 Iteration: 5 Train loss: 0.083
Epoch: 3/30 Iteration: 10 Train loss: 0.060
Epoch: 4/30 Iteration: 15 Train loss: 0.047
Epoch: 6/30 Iteration: 20 Train loss: 0.038
Epoch: 8/30 Iteration: 25 Train loss: 0.004
Epoch: 9/30 Iteration: 30 Train loss: 0.008
Epoch: 11/30 Iteration: 35 Train loss: 0.016
Epoch: 13/30 Iteration: 40 Train loss: 0.000
Epoch: 14/30 Iteration: 45 Train loss: 0.008
Epoch: 16/30 Iteration: 50 Train loss: 0.007
Epoch: 18/30 Iteration: 55 Train loss: 0.000
Epoch: 19/30 Iteration: 60 Train loss: 0.006
Epoch: 21/30 Iteration: 65 Train loss: 0.008
Epoch: 23/30 Iteration: 70 Train loss: 0.002
Epoch: 24/30 Iteration: 75 Train loss: 0.002
Epoch: 26/30 Iteration: 80 Train loss: 0.001
Epoch: 28/30 Iteration: 85 Train loss: 0.000
Epoch: 29/30 Iteration: 90 Train loss: 0.001


# Testing model

In [46]:
import csv

In [47]:
features_test = getFeatures(texts_test, labels_test, vocab_to_int)
print(features_test[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  886  120   67 1753   40   99  193  600   21    6   38  346   86  377
   55  110  398    2   41   11   13   84 1903   46  352 1080 9310    0
   69 

In [48]:
test_acc = []
title = np.column_stack(('text', 'prediction', 'label'))
out_path = saveDataPath + 'prediction.csv'

with open(out_path, 'w') as f:
    csv.writer(f).writerows(title)
    
    with tf.Session(graph=graph) as sess:
        saver.restore(sess, tf.train.latest_checkpoint(saveDataPath))
        test_state = sess.run(stack_cells.zero_state(batch_size, tf.float32))
        for ii, (text, x, y) in enumerate(get_batches_test(texts_test, features_test, labels_test, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 1,
                    initial_state: test_state}
            batch_acc, test_state, batch_cor = sess.run([accuracy, final_state, correct_pred], feed_dict=feed)
            test_acc.append(batch_acc)
            predict_label = []
            for i in range(len(y)):
                predict_label.append(y[i] if batch_cor[i] == True else 1 - y[i])
            csv.writer(f).writerows(np.column_stack((np.array(text), predict_label, y)))
        print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from /Users/SamZhang/Documents/Capstone/Models/runs/lstmmodel/1/sentiment.ckpt
Test accuracy: 0.931


# Evaluation and Noise Reduction

In [51]:
import ES_interface as esi

esi.metric(dataset + '_' + str(trainingSection), saveDataPath + 'prediction.csv')

362 33 4287 310
accuracy : 0.931 
precision : 0.916 
recall : 0.539 
f1 : 0.679 

327 2 4318 345
accuracy : 0.930 
precision : 0.994 
recall : 0.487 
f1 : 0.653 

